# Extracting and Visualizing Stock Data

# Description

Extracting essential data from a dataset and displaying it is a necessary part of data science; therefore individuals can make correct decisions based on the data. In this assignment, you will extract some stock data, you will then display this data in a graph.



# Table of Contents

# Define a Function that Makes a Graph
Question 1: Use yfinance to Extract Stock Data
    
Question 2: Use Webscraping to Extract Tesla Revenue Data
    
Question 3: Use yfinance to Extract Stock Data
    
Question 4: Use Webscraping to Extract GME Revenue Data
    
Question 5: Plot Tesla Stock Graph
    
Question 6: Plot GameStop Stock Graph
    
Estimated Time Needed: 30 min

In [1]:
!pip install yfinance
#!pip install pandas
#!pip install requests
!pip install bs4
#!pip install plotly

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23907 sha256=3a811715a69a90853450a893ab0f885ab297dd6e3998c751d0f1acc66fc7f3ab
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
  Created wheel for multitasking: filename=multitasking-0.0.9-py3-none-any.whl size=8367 sha256=db47c41f0c8128185aaac316becc0a9ae6d5b7d9207ae5334eab0b34d5874dd9
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/ae/25/47/4d68431a7ec1b6c4b5233365934b74c1d4e665bf5f968d363a


In [2]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define Graphing Function

In this section, we define the function make_graph. You don't have to know how the function works, you should only care about the inputs. It takes a dataframe with stock data (dataframe must contain Date and Close columns), a dataframe with revenue data (dataframe must contain Date and Revenue columns), and the name of the stock.

In [3]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

# Question 1: Use yfinance to Extract Stock Data

Using the Ticker function enter the ticker symbol of the stock we want to extract data on to create a ticker object. The stock is Tesla and its ticker symbol is TSLA.

In [4]:
Tesla=yf.Ticker("TSLA")
Tesla

yfinance.Ticker object <TSLA>

Using the ticker object and the function history extract stock information and save it in a dataframe named tesla_data. Set the period parameter to max so we get information for the maximum amount of time.

In [5]:
tesla_data=Tesla.history(period="max")

Reset the index using the reset_index(inplace=True) function on the tesla_data DataFrame and display the first five rows of the tesla_data dataframe using the head function. Take a screenshot of the results and code from the beginning of Question 1 to the results below.

In [6]:
tesla_data.reset_index(inplace=True)
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29,3.800,5.000,3.508,4.778,93831500,0,0.0
1,2010-06-30,5.158,6.084,4.660,4.766,85935500,0,0.0
2,2010-07-01,5.000,5.184,4.054,4.392,41094000,0,0.0
3,2010-07-02,4.600,4.620,3.742,3.840,25699000,0,0.0
4,2010-07-06,4.000,4.000,3.166,3.222,34334500,0,0.0


In [7]:
tesla_data.dtypes

Date            datetime64[ns]
Open                   float64
High                   float64
Low                    float64
Close                  float64
Volume                   int64
Dividends                int64
Stock Splits           float64
dtype: object

# Question 2: Use Webscraping to Extract Tesla Revenue Data

Use the requests library to download the webpage https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue. Save the text of the response as a variable named html_data.

In [8]:
url="https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
html_data=requests.get(url).text

Parse the html data using beautiful_soup.

In [10]:
soup=BeautifulSoup(html_data, "html5lib")

Using beautiful soup extract the table with Tesla Quarterly Revenue and store it into a dataframe named tesla_revenue. The dataframe should have columns Date and Revenue.

Click here if you need help locating the table

In [11]:
response = requests.get('https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue')
soup = BeautifulSoup(response.text, 'lxml')

all_tables = soup.find_all('table', attrs={'class': 'historical_data_table table'})

tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])

for table in all_tables:
    if table.find('th').getText().startswith("Tesla Quarterly Revenue"):
        for row in table.find_all("tr"):
            col = row.find_all("td")  
            if len(col) == 2: 
                date = col[0].text
                revenue = col[1].text.replace('$', '').replace(',', '')
                tesla_revenue = tesla_revenue.append({"Date": date, "Revenue": revenue}, ignore_index=True)

#tesla_revenue = tesla_revenue.apply(pd.to_numeric, errors='coerce')
#tesla_revenue = tesla_revenue.dropna()

print(tesla_revenue)

          Date Revenue
0   2021-06-30   11958
1   2021-03-31   10389
2   2020-12-31   10744
3   2020-09-30    8771
4   2020-06-30    6036
5   2020-03-31    5985
6   2019-12-31    7384
7   2019-09-30    6303
8   2019-06-30    6350
9   2019-03-31    4541
10  2018-12-31    7226
11  2018-09-30    6824
12  2018-06-30    4002
13  2018-03-31    3409
14  2017-12-31    3288
15  2017-09-30    2985
16  2017-06-30    2790
17  2017-03-31    2696
18  2016-12-31    2285
19  2016-09-30    2298
20  2016-06-30    1270
21  2016-03-31    1147
22  2015-12-31    1214
23  2015-09-30     937
24  2015-06-30     955
25  2015-03-31     940
26  2014-12-31     957
27  2014-09-30     852
28  2014-06-30     769
29  2014-03-31     621
30  2013-12-31     615
31  2013-09-30     431
32  2013-06-30     405
33  2013-03-31     562
34  2012-12-31     306
35  2012-09-30      50
36  2012-06-30      27
37  2012-03-31      30
38  2011-12-31      39
39  2011-09-30      58
40  2011-06-30      58
41  2011-03-31      49
42  2010-12

Execute the following line to remove the comma and dollar sign from the Revenue column.

In [12]:
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"")

Execute the following lines to remove an null or empty strings in the Revenue column.

In [14]:
tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

Display the last 5 row of the tesla_revenue dataframe using the tail function. Take a screenshot of the results.

In [15]:
tesla_revenue.tail()

,Date,Revenue
43,2010-09-30,31
44,2010-06-30,28
45,2010-03-31,21
47,2009-09-30,46
48,2009-06-30,27


# Question 3: Use yfinance to Extract Stock Data

Using the Ticker function enter the ticker symbol of the stock we want to extract data on to create a ticker object. The stock is GameStop and its ticker symbol is GME.

In [16]:
GameStop=yf.Ticker("GME")
GameStop

yfinance.Ticker object <GME>

Using the ticker object and the function history extract stock information and save it in a dataframe named gme_data. Set the period parameter to max so we get information for the maximum amount of time.

In [17]:
game_data=GameStop.history(period="Max")

Reset the index using the reset_index(inplace=True) function on the gme_data DataFrame and display the first five rows of the gme_data dataframe using the head function. Take a screenshot of the results and code from the beginning of Question 3 to the results below.

In [18]:
game_data.reset_index(inplace=True)
game_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13,6.480513,6.773399,6.413183,6.766666,19054000,0.0,0.0
1,2002-02-14,6.850831,6.864296,6.682506,6.733003,2755400,0.0,0.0
2,2002-02-15,6.733001,6.749833,6.632006,6.699336,2097400,0.0,0.0
3,2002-02-19,6.665671,6.665671,6.312189,6.430017,1852600,0.0,0.0
4,2002-02-20,6.463681,6.648838,6.413183,6.648838,1723200,0.0,0.0


# Question 4: Use Webscraping to Extract GME Revenue Data

Use the requests library to download the webpage https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue. Save the text of the response as a variable named html_data.



In [19]:
html_data = requests.get("https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue")

Parse the html data using beautiful_soup.

In [20]:
soup = BeautifulSoup(html_data.text, 'lxml')

In [21]:
all_tables = soup.find_all('table', attrs={'class': 'historical_data_table table'})

game_revenue = pd.DataFrame(columns=["Date", "Revenue"])

for table in all_tables:
    if table.find('th').getText().startswith("GameStop Quarterly Revenue"):
        for row in table.find_all("tr"):
            col = row.find_all("td")  
            if len(col) == 2: 
                date = col[0].text
                revenue = col[1].text.replace('$', '').replace(',', '')
                game_revenue = game_revenue.append({"Date": date, "Revenue": revenue}, ignore_index=True)

#tesla_revenue = tesla_revenue.apply(pd.to_numeric, errors='coerce')
#tesla_revenue = tesla_revenue.dropna()

print(game_revenue)

          Date Revenue
0   2021-04-30    1277
1   2021-01-31    2122
2   2020-10-31    1005
3   2020-07-31     942
4   2020-04-30    1021
..         ...     ...
61  2006-01-31    1667
62  2005-10-31     534
63  2005-07-31     416
64  2005-04-30     475
65  2005-01-31     709

[66 rows x 2 columns]


Using beautiful soup extract the table with GameStop Quarterly Revenue and store it into a dataframe named gme_revenue. The dataframe should have columns Date and Revenue. Make sure the comma and dollar sign is removed from the Revenue column using a method similar to what you did in Question 2.

Click here if you need help locating the table

In [23]:
all_tables = soup.find_all('table', attrs={'class': 'historical_data_table table'})

game_revenue = pd.DataFrame(columns=["Date", "Revenue"])

for table in all_tables:
    if table.find('th').getText().startswith("GameStop Quarterly Revenue"):
        for row in table.find_all("tr"):
            col = row.find_all("td")  
            if len(col) == 2: 
                date = col[0].text
                revenue = col[1].text.replace('$', '').replace(',', '')
                game_revenue = game_revenue.append({"Date": date, "Revenue": revenue}, ignore_index=True)


print(game_revenue)

          Date Revenue
0   2021-04-30    1277
1   2021-01-31    2122
2   2020-10-31    1005
3   2020-07-31     942
4   2020-04-30    1021
..         ...     ...
61  2006-01-31    1667
62  2005-10-31     534
63  2005-07-31     416
64  2005-04-30     475
65  2005-01-31     709

[66 rows x 2 columns]


Display the last five rows of the gme_revenue dataframe using the tail function. Take a screenshot of the results.

In [24]:
game_revenue.tail()

,Date,Revenue
61,2006-01-31,1667
62,2005-10-31,534
63,2005-07-31,416
64,2005-04-30,475
65,2005-01-31,709


# Question 5: Plot Tesla Stock Graph